# Análisis de sueldos con Python

## Data

* [2021.2 - sysarmy - Encuesta de remuneración salarial Argentina
](https://docs.google.com/spreadsheets/d/1-ZRznwS4TK74o90aOiCKS5SiXxUQ2buN1wxZIMHQmzQ/edit#gid=557755710) 
* [2021.2 - sysarmy - Encuesta de remuneración salarial Latam
](https://docs.google.com/spreadsheets/d/1BkBNt1MHVS7DeIlpgmK9l6krtSQ5t_olRhlcyxMeKy0/edit#gid=557755710)

1. Descargar, leer y unir en una sola estructura de datos. ¿Qué estructura elegiste y por qué?

In [ ]:
# En este primer punto se traerán los datos de los dos archivos para trabajar.
# Se ordenarán de tal forma que simplifique el trabajo posterior, así como la normalizacion de sus valores.

In [ ]:
# A continuación la importación de la encuesta de Latam y la definición de su estructura que será listas de diccionarios.

In [40]:
import csv

list_keys = []
list_values = []
list_dict_info_latam = []                                     # acá se guarda la info de los dict de Latam

with open('2021.2 - sysarmy - Encuesta de remuneración salarial Latam - Latam.tsv', encoding="utf8") as s_latam:
    latam_reader = csv.reader(s_latam, delimiter='\t')
    for index, line in enumerate(latam_reader):
        #if index > 9:
            #print(f'{latam_line}')
        # Líneas 0 a 9 vacías
        # Líneas 10  -> preguntas
        # Líneas 11+ -> respuestas
        if index == 10:
            list_keys = line
            list_keys[0] = 'País'                              # La pregunta Estoy trabajando en se pasa a llamar País
            dict_base = dict()
            for key in list_keys:
                dict_base[key] = ''
            #print(line)
        elif index >= 11:
            list_values = line
            dict_info = dict_base.copy()
            for i, key in enumerate(dict_info.keys()):
                dict_info[key] = list_values[i]
            #print(dict_info)
            dict_info['Ciudad'] = dict_info[dict_info['País']]  # Se agrega una pregunta Ciudad con respuesta variable
            
            for i, key in enumerate(list_keys[1:19]):           # Para borrar las columnas de los países con sus regiones
                dict_info.pop(key)
            #print(dict_info)
            
            list_dict_info_latam.append(dict_info)
        



In [ ]:
# A continuación la importación de la encuesta de Argentina y la definición de la estructura listas de diccionario.

In [39]:
import csv

list_keys = []
list_values = []
list_dict_info_arg = []                           # Acá se guarda la info de los dict de Argentina

with open('2021.2 - sysarmy - Encuesta de remuneración salarial Argentina - Argentina.tsv', encoding="utf8") as s_arg:
    arg_reader = csv.reader(s_arg, delimiter='\t')
    for index, line in enumerate(arg_reader):
        #if index > 8:
            #print(f'{arg_line}')
        # Líneas 0 a 8 vacías
        # Líneas 9   -> preguntas
        # Líneas 10+ -> respuestas
        if index == 9:
            list_keys = line
            list_keys[0] = 'Ciudad'                #La pregunta Dónde estás trabajando se pasa a llamar Ciudad
            dict_base = dict()
            for key in list_keys:
                dict_base[key] = ''
            #print(line)
        elif index >= 10:
            list_values = line
            dict_info = dict_base.copy()
            for i, key in enumerate(dict_info.keys()):
                dict_info[key] = list_values[i]
            #print(dict_info)
            dict_info['País'] = 'Argentina'          # Se agrega una pregunta País con respuesta Argentina
            
            list_dict_info_arg.append(dict_info)
                

In [ ]:
# A continuación seran unidas las dos estructuras en una sola. Argentina se agregará a Latam

In [8]:
list_all = list_dict_info_latam + list_dict_info_arg

In [13]:
print(len(list_all))

6869


In [10]:
print(len(list_dict_info_latam))

429


In [11]:
print(len(list_dict_info_arg))

6440


In [ ]:
# A continuación se normalizarán los elementos

In [61]:
# Función de normalización:

def normalize_record(dict_record):
    dict_result = {}
    for i, key in enumerate(dict_record.keys()):
    
        if key == 'Ciudad':
            if dict_record[key] == '':
                dict_result[key] = 'SIN CIUDAD'
            else:
                dict_result[key] = dict_record[key]
        else:
            dict_result[key] = dict_record[key]
      
    return dict_result

list_all_normalized = []
for i, key in enumerate(list_all):
    list_all_normalized.append(normalize_record(list_all[i]))

#Para verificar que se cambió bien el registro    
print(list_all[0])
print('---------------------------------------------------------------------------------')
print(list_all_normalized[0])
    

{'País': 'Bolivia', 'Tipo de contrato': 'Full-Time', 'Salario mensual o retiro BRUTO (en tu moneda local)': '13500', 'Salario mensual o retiro NETO (en tu moneda local)': '12000', 'Pagos en dólares': '', '¿Cuál fue el último valor de dólar que tomaron?': '', '¿Qué tan conforme estás con tu sueldo?': '3', 'Cómo creés que está tu sueldo con respecto al último semestre': '3', 'Recibís algún tipo de bono': 'Menos de un sueldo', 'A qué está atado el bono': 'Performance de la compañía', '¿Tuviste ajustes por inflación durante 2021?': 'No', '¿De qué % fue el ajuste total?': '0', '¿En qué mes fue el último ajuste?': '0', 'Trabajo de': 'SysAdmin / DevOps / SRE', 'Años de experiencia': '6', 'Años en la empresa actual': '14', 'Años en el puesto actual': '6', '¿Gente a cargo?': '2', 'Plataformas': 'Amazon Web Services, Azure, Docker, Linux, OpenStack', 'Lenguajes de programación o tecnologías.': 'Bash/Shell', 'Frameworks, herramientas y librerías': 'Ninguno de los anteriores', 'Bases de datos': 'A

In [ ]:
#Definiciones de funciones a utilizar

In [68]:
# funcion que devuelve el porcentaje

def percentage(total,quantity):
    return round(quantity * 100  / total, 2)


# funcion que calcula la paticpuacion ordenada desceiente de una clave

def participation_per_key_DESC(list_to_analyze, key):
    dict_result_Participacion_Region = {}

    for record in list_to_analyze:
        if record[key] in dict_result_Participacion_Region:
            # Si la ciudad ya esta en el diccionario resultado incremento el contador
            dict_result_Participacion_Region[record[key]] += 1 
        else:
            # Si la ciudad NO esta en el diccionario resultado inicializo el contador en 1
            dict_result_Participacion_Region[record[key]] = 1 

    register_total = len(list_all_normalized)

    return sorted(dict_result_Participacion_Region.items(), key=lambda x: x[1], reverse=True)

2. Printear porcentaje de participación por región, ordenado de mayor a menor. Ej: 

```
- Ciudad Autónoma de Buenos Aires - 59.4%
- Formosa - 0.04%
```

In [69]:
sorted_tuples_Participacion_Region = participation_per_key_DESC(list_all_normalized, 'Ciudad')
for tuple_record in sorted_tuples_Participacion_Region:
    print(f' - {tuple_record[0]} - {percentage(register_total,tuple_record[1])}%')


 - Ciudad Autónoma de Buenos Aires - 51.71%
 - Provincia de Buenos Aires - 10.1%
 - GBA - 9.2%
 - Córdoba - 7.8%
 - Santa Fe - 5.82%
 - Mendoza - 2.18%
 - Montevideo - 1.7%
 - Entre Ríos - 0.96%
 - Río Negro - 0.76%
 - Tucumán - 0.76%
 - Neuquén - 0.68%
 - Misiones - 0.61%
 - Santiago - 0.58%
 - Chaco - 0.57%
 - México DF - 0.47%
 - Lima - 0.45%
 - Salta - 0.42%
 - San Juan - 0.35%
 - Corrientes - 0.35%
 - Bogotá - 0.28%
 - Chubut - 0.28%
 - Jujuy - 0.28%
 - Medellín - 0.25%
 - Quito - 0.23%
 - La Rioja - 0.23%
 - San Luis - 0.17%
 - Tierra del Fuego - 0.17%
 - Santiago de Cali - 0.13%
 - Ciudad Guatemala - 0.12%
 - La Pampa - 0.12%
 - Santiago del Estero - 0.12%
 - Guadalajara - 0.1%
 - Guayaquil - 0.09%
 - Asunción - 0.09%
 - Santa Cruz de la Sierra - 0.07%
 - Santa Cruz - 0.07%
 - Caracas - 0.07%
 - Querétaro - 0.06%
 - Cochabamba - 0.04%
 - Valparaíso - 0.04%
 - Tunja - 0.04%
 - Manizales - 0.04%
 - San José - 0.04%
 - Loja - 0.04%
 - Monterrey - 0.04%
 - Veracruz - 0.04%
 - Puebla

3. Printear porcentaje de participación por tipo de rol, ordenado de mayor a menor. Ej:  
`- Developer - 39.11 %`

In [70]:
sorted_tuples_Participacion_Rol = participation_per_key_DESC(list_all_normalized, 'Trabajo de')
for tuple_record in sorted_tuples_Participacion_Rol:
    print(f' - {tuple_record[0]} - {percentage(register_total,tuple_record[1])}%')


 - Developer - 40.69%
 - SysAdmin / DevOps / SRE - 11.09%
 - Technical Leader - 6.16%
 - QA / Tester - 4.29%
 - Manager / Director - 4.28%
 - Project Manager - 3.09%
 - HelpDesk - 2.62%
 - BI Analyst / Data Analyst - 2.45%
 - Architect - 2.23%
 - Consultant - 1.92%
 - Data Engineer - 1.76%
 - Business Analyst - 1.69%
 - Networking - 1.54%
 - UX - 1.5%
 - Data Scientist - 1.48%
 - Infosec - 1.43%
 - Recruiter / HR - 1.24%
 - Product Manager - 1.21%
 - DBA - 0.71%
 - Scrum Master - 0.66%
 - Designer - 0.64%
 - VP / C-Level - 0.55%
 - Sales / Pre-Sales - 0.42%
 - Middleware - 0.17%
 - Finance - 0.17%
 - Technical Writer - 0.16%
 - Storage / Backup - 0.12%
 - Analista funcional - 0.09%
 - Docente - 0.07%
 - Product Owner - 0.06%
 - Agile Coach - 0.04%
 - Software Engineer - 0.04%
 - analista funcional - 0.04%
 - Analista Funcional - 0.04%
 - DevOps - 0.04%
 - Coordinador - 0.04%
 - RPA - 0.04%
 - Analista - 0.03%
 - Team leader - 0.03%
 - Support Engineer - 0.03%
 - People Analytics - 0.03

4. Printear porcentaje de respuesta para las preguntas: 
* ¿Contribuís a proyectos Open Source?
* ¿Programás por hobbie?

In [76]:
sorted_tuples_Participacion_Open_Source = participation_per_key_DESC(list_all_normalized, '¿Contribuís a proyectos open source?')
print('¿Contribuís a proyectos open source?')
for tuple_record in sorted_tuples_Participacion_Open_Source:
    print(f' - {tuple_record[0]} - {percentage(register_total,tuple_record[1])}%')

¿Contribuís a proyectos open source?
 -  - 49.44%
 - No - 42.99%
 - Sí - 7.57%


In [77]:
sorted_tuples_Participacion_Hobbie = participation_per_key_DESC(list_all_normalized, '¿Programás como hobbie?')
print('¿Programás por hobbie?')
for tuple_record in sorted_tuples_Participacion_Hobbie:
    print(f' - {tuple_record[0]} - {percentage(register_total,tuple_record[1])}%')

¿Programás por hobbie?
 -  - 49.44%
 - Sí - 28.34%
 - No - 22.22%


5. Printear porcentaje por seniority según años de experiencia, el mapeo es:
```
Junior: de 0 hasta 2 años.
Semi-Senior: de 2 años inclusive hasta 5 años.
Senior: desde 5 años inclusive.
``` 

6. Printear porcentaje de personas encuestadas por años en la compañía actual y por años en el puesto actual.

7. Printear porcentajes de nivel de educación formal y estado, es decir: % educación secundaria, terciaria, universitaria, postgrado, doctorado, postdoctorado, completo, incompleto y en curso para cada uno.

8. Formatear las carreras universitarias:
- Nombres capitalizados
- Reemplazar vocales con tilde por vocales sin tilde.
- Reemplazar `ñ` por `n`
- *lic*, *lic.* por Licenciatura
- *tec*, *tec.* por Tecnucatura
* *cs, *cs.* por Ciencias
* *ed, ed.* por Educación
* Transformaciones que se consideren necesarias

Printear porcentaje según carrera

9. Printear porcentaje de respuesta a las preguntas:
- ¿Realizaste cursos de especialización?
- Si realizaste, ¿Quién pagó por los cursos de especialización?

10. Printear porcentaje por identidad de género y personas con discapacidad

11. Salarios: calcular la mediana salarial para cada una de las siguientes categorías:
- Salario según región
- Salario por rol
- Salario por experiencia
- Salario por nivel de formación
- Salario por carrera
- Salario por tecnología
- Salario por lenguaje de programación
- Salario por género

12. Printear porcentaje y salario según tipos de contrato .

13. En base a los resultados obtenidos confeccionar conclusiones respecto a:

- Rol vs sueldos
- Nivel de educación alcanzada vs sueldos
- Género vs sueldos
- Tecnologías populares

14. Generar gráficos de barra para mostrar los resultados mencionados anteriormente usando strings, ej:

```
developer | -----------------------------------------------
sysadmin  | ---------------------------
QA        | ------------------
          | ..................5%.......10%...............40%
```